# Pop Grid Compare

Goal of this notebook is to load all availabe popgrid date products with the same spatial resolution, projections, and CRS, stack them, and indentify areas of geographic variance.

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio 
import rioxarray as rio

In [ ]:
# File paths and FN
data_in = '/Users/cascade/Github/PopGridCompare/data/raw/'
data_out = '/Users/cascade/Github/PopGridCompare/data/interim/'

wp_path = data_in+'WorldPop16/ppp_2016_1km_Aggregated.tif'
ls_path = data_in+'LandScan-Global-2015/lspop2015/w001001.adf'
esri_path = data_in+'ESRI_WPE_2016_Pop/WPE_1KM_2016_Pop.tif'

## Open files and check meta data

In [ ]:
# World Pop : width = 43200 height = 18720 epsg = 4326
wp = rio.open_rasterio(wp_path)
wp

In [ ]:
# ESRI : width = 40074 height = 14285 epsg = 4326
esri = rio.open_rasterio(esri_path)
esri

In [ ]:
# LS : width = 43200 height = 21600 epsg = 4326
ls = rio.open_rasterio(ls_path)
ls


# Reset zero values

In [ ]:
print('World Pop na value is', wp.data[0][0][0])
print('LandScan na value is', ls.data[0][0][0])
print('ESRI na value is', esri.data[0][0][0])

In [ ]:
#### Try setting all NAs and 0s to Zero
wp.data = np.where(wp.data < 1, 0, wp.data)
ls.data = np.where(ls.data < 1, 0, ls.data)
esri.data = np.where(esri.data < 1, 0, esri.data)

In [ ]:
print('World Pop na value is', wp.data[0][0][0])
print('LandScan na value is', ls.data[0][0][0])
print('ESRI na value is', esri.data[0][0][0])

## Test Matching and Reprojecting

trying: https://corteva.github.io/rioxarray/html/examples/reproject_match.html

In [ ]:
# Check NA values
print(esri.attrs['_FillValue'])
print(ls.attrs['_FillValue'])
print(wp.attrs['_FillValue'])

In [ ]:
# Change NA values
na_val = -9999
esri.attrs['_FillValue'] = na_val
ls.attrs['_FillValue'] = na_val
wp.attrs['_FillValue'] = na_val

In [ ]:
# Check NA values
print(esri.attrs['_FillValue'])
print(ls.attrs['_FillValue'])
print(wp.attrs['_FillValue'])

In [ ]:
# Match WorldPop/ESRI to LandSan write it out

wp_match = wp.rio.reproject_match(ls)
esri_match = esri.rio.reproject_match(ls)

In [ ]:
# Check NA valuesb
print(esri_match.attrs['_FillValue'])
print(wp.attrs['_FillValue'])
print(ls.attrs['_FillValue'])


In [ ]:
# Try writing them and then checking them

esri_match.rio.to_raster(data_out+'ESRI16_matched.tif')
wp_match.rio.to_raster(data_out+'WP16_matched.tif')
#ls.rio.to_raster(data_out+'LS15_matched.tif')



In [ ]:
ls.rio.to_raster(data_out+'LS15_matched.tif')

## Subtract 'EM

In [ ]:
# This produces strange results

wp_ls = wp - ls_match
wp_esri = wp - esri_match
ls_esri = ls_match - esri_match

In [ ]:
# write them to files
wp_ls.rio.to_raster(data_out+'wp_ls.tif')
wp_esri.rio.to_raster(data_out+'wp_esri.tif')
ls_esri.rio.to_raster(data_out+'ls_esri.tif')

In [ ]:
rgb_array = xr.concat([wp, esri_match, ls_match], dim = 'band')

In [ ]:
#rgb_array.rio.to_raster(data_out+'all_rgb.tif')